# C1.1 Evaluation on Object Detection

## Introduction

- Task: Evaluate the average precision of the data collected from SafeBench.

- Submission: the plotted P-R curve, as well as the average precision at different IoU threshold level.

In [ ]:
import joblib
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
!git clone https://github.com/HenryLHH/24784-c1-data.git
!cp /content/24784-c1-data/results.pkl ./
!rm -r /content/24784-c1-data/


### Load the Detection Results from SafeBench_v2

In [ ]:
inputs = joblib.load('results.pkl')  # move the files to your current folder
inputs.keys() # dict_keys(['image_id', 'predicted_class', 'ground_truth_bbox', 'predicted_bbox', 'conf_scores', 'num_labels'])

dict_keys(['image_id', 'predicted_class', 'ground_truth_bbox', 'predicted_bbox', 'conf_scores', 'num_labels'])

### Compute the IoU

In [ ]:
def box_area(box):
    """ Compute the box area, given all the vertices
    # Arguments
        box:  (4, N) ndarray
    # Returns
        areas: (N, ) ndarray
    """
    areas = ...   # TODO, compute the rectangle areas based on the vertices input
    return areas

def box_iou(box1, box2):
    '''
      Compute the iou between box1 and box2
      ONLY consider the SINGLE ground truth, which is a simplified case
      box1: (N, 4) ndarray
      box2: (N, 4) ndarray
      return: (N, ) iou scores
    '''
    eps = 1e-7

    a1, a2 = np.split(box1, 2, axis=1)
    b1, b2 = np.split(box2, 2, axis=1)

    inter =  ...                                                          # TODO, compute the area of intersectinos
    union = ...                                                           # TODO, compute the areas of union
    return inter / (union + eps)


In [ ]:
# TODO: get the box iou scores from the function you implemented above
iou_scores = box_iou(inputs['ground_truth_bbox'], inputs['predicted_bbox'])

# Build your dataframe from the dictionary
input_dict = {
        'image_id': inputs['image_id'],
        'predicted_class': inputs['predicted_class'],
        'conf_scores': inputs['conf_scores'],
        'iou_scores': iou_scores,
      }
df = pd.DataFrame.from_dict(input_dict)
df

In [ ]:
df = df[df.conf_scores >= 0] # drop all the frames without detection (conf scores = -1)

# get all the detection results of stop signs
df_stopsign = df.loc[(df.predicted_class=='stopsign')]
df_stopsign

## Compute the Avereage Precision

In [ ]:
def interp_ap(recall, precision, method = 'interp'):
    """ Compute the average precision, given the recall and precision curves
    # Arguments
        recall:    The recall curve (list)
        precision: The precision curve (list),
        methods: 'continuous', 'interp'
    # Returns
        Average precision, precision curve, recall curve
    """

    # TODO: Append sentinel values to beginning and end
    # Recall should start with 0.0 and end with 1.0
    # Precision should start with 1.0 and end with 0.0

    appended_recall = np.concatenate(([0.0], recall, [1.0]))
    appended_prec_input = np.concatenate(([1.0], precision, [0.0]))

    # Compute the precision envelope
    appended_prec = np.flip(np.maximum.accumulate(np.flip(appended_prec_input)))   # get the p(r) = \max_{r<=r'} p(r')

    # Integrate area under curve
    if method == 'interp':
        x = np.linspace(0, 1, 101)  # 101-point interp (COCO)
        ap = np.trapz(np.interp(x, appended_recall, appended_prec), x)      # get the average precision based on the areas under interpolated curves
    else:  # 'continuous'
        i = np.where(appended_recall[1:] != appended_recall[:-1])[0]  # points where x axis (recall) changes
        ap = np.sum((appended_recall[i + 1] - appended_recall[i]) * appended_prec[i + 1])  # area under curve

    return ap, appended_prec_input, appended_prec, appended_recall



In [ ]:
def compute_ap(df, num_gt, iou_thres):
    """ Compute the average precision, given the recall and precision curves
    # Arguments
        df:         DataFrame inputs containing predicted classes, iou_scores, and confidence
        num_gt:     The precision curve (list),
        iou_thres:  IoU threshold of True Positive (TP) detection
    # Returns
        Average precision, precision curve, recall curve
    """
    df = ...                                                           # TODO sort all the data with confidence scores 'conf_scores'
    tp_fp = np.arange(1, len(df)+1, 1)
    tp = ((df['iou_scores'] >= iou_thres) & (df['predicted_class'] == 'stopsign')).cumsum().values    # get the true positive sets
    precision = tp / tp_fp      # array (N, )
    recall = tp / num_gt        # array (N, )

    ap, prec_raw, prec, recall = interp_ap(recall, precision)           # get the AP and returned curve for plot

    return ap, prec_raw, prec, recall

## Visualization

In [ ]:
def plot_pr_curves(recall, prec_raw, prec):
    """ Visualize the P-R curves
        Visualize the areas under P-R curves
    """
    plt.figure()
    plt.plot(recall, prec_raw)
    plt.plot(recall, prec)
    plt.fill_between(recall, np.zeros_like(recall), prec, color='orange', alpha=0.2)
    plt.legend(['Raw curve', 'Precision Envelope', 'Average Precision'])
    plt.title('P-R Curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')

## Execute the AP calculation

In [ ]:
# Traverse over IoU threshold, get AP@[0.5:0.1:0.9]

for iou_thres in np.arange(0.5, 1.0, 0.1):
    ap, prec_raw, prec, recall = compute_ap(df, inputs['num_labels'], iou_thres)
    print('IoU Threshold: {:.2f}'.format(iou_thres), '  |    AP@{:.2f}'.format(iou_thres), ap)
    plot_pr_curves(recall, prec_raw, prec)